In [21]:
#导入需要的库
import numpy as np
import pretty_midi
import os
from operator import itemgetter, attrgetter

In [22]:
def data_process(midi_file, npy_file):
    #加载npy数据
    p_data = np.load(npy_file)
    for d in range(p_data.shape[0]):
        p_data[d][0] = float(p_data[d][0])/10000000
        p_data[d][1] = float(p_data[d][1])/10000000
    #加载midi数据
    midi_data = pretty_midi.PrettyMIDI(midi_file)
    for inst in midi_data.instruments:
        instrument = inst
    #把midi的音符对齐到0.005s的倍数
    for iii in range(len(instrument.notes)):
        ob_s = instrument.notes[iii].start
        ob_e = instrument.notes[iii].end
        instrument.notes[iii].start = ((ob_s * 1000000) - (ob_s * 1000000)%(0.005 * 1000000))/1000000
        instrument.notes[iii].end = ((ob_e * 1000000) - (ob_e * 1000000)%(0.005 * 1000000))/1000000
    #让每个音头尾相接
    for idx in range(len(instrument.notes) - 1):
        instrument.notes[idx].end = instrument.notes[idx + 1].start
    #让第一个note的开始时间是它属于的音素的开始时间
    first_note_start = instrument.notes[0].start
    for i_1 in range(p_data.shape[0]):
        if float(p_data[i_1][0]) < first_note_start and first_note_start <= float(p_data[i_1][1]):
            instrument.notes[0].start = float(p_data[i_1][0])
            break
        else:
            pass
    #让最后一个note的结束时间是它属于的音素的结束时间
    last_note_end = instrument.notes[len(instrument.notes) - 1].end
    for i_2 in range(p_data.shape[0]):
        if float(p_data[i_2][0]) < last_note_end and last_note_end <= float(p_data[i_2][1]):
            instrument.notes[len(instrument.notes) - 1].end = float(p_data[i_2][1])
    #确立一个音色
    cello_program = pretty_midi.instrument_name_to_program('Cello')
    #建立3个空乐器
    cello = pretty_midi.Instrument(program = cello_program)
    cello_middle = pretty_midi.Instrument(program = cello_program)

    jj = pretty_midi.Instrument(program = cello_program)

    for k in range(p_data.shape[0]):
        for n_1 in instrument.notes:
            if n_1.start > float(p_data[k][0]) and n_1.end <= float(p_data[k][1]):
                t_1_n = pretty_midi.Note(velocity = 127, pitch = n_1.pitch, start = n_1.start, end = n_1.end)
                cello.notes.append(t_1_n)
            else:
                pass
    for n_2 in instrument.notes:
        op = 0
        for n_3 in cello.notes:
            if n_2.start == n_3.start:
                op += 1
            else:
                pass
        if op == 0:
            cello_middle.notes.append(n_2)
    for io,n_4 in enumerate(cello_middle.notes):
        for k_2 in range(p_data.shape[0]):
            pho_s = float(p_data[k_2][0])
            pho_e = float(p_data[k_2][1])
            if pho_s < n_4.start and pho_e <= n_4.end and pho_e > n_4.start:
                t_2_n = pretty_midi.Note(velocity =127,pitch = n_4.pitch, start = n_4.start,end = pho_e)
                cello.notes.append(t_2_n)
            elif pho_s >= n_4.start and pho_e <= n_4.end:
                t_3_n = pretty_midi.Note(velocity = 127, pitch = n_4.pitch, start = pho_s, end = pho_e)
                cello.notes.append(t_3_n)
            elif pho_s >= n_4.start and pho_s <= n_4.end and pho_e >= n_4.end:
                t_4_n = pretty_midi.Note(velocity = 127, pitch = n_4.pitch, start = pho_s, end = n_4.end)
                cello.notes.append(t_4_n)
    a = sorted(cello.notes, key = attrgetter('start'))
    kiki = []
    for k_3 in range(p_data.shape[0]):
        pho_s = float(p_data[k_3][0])
        pho_e = float(p_data[k_3][1])
        pho = p_data[k_3][2]
        for n_5 in a:
            if n_5.start >= pho_s and n_5.end <= pho_e and pho != 'pau' and pho != 'br':
                kiki.append([n_5, pho])
    peng = []
    for xx,gg in enumerate(kiki):
        if (gg[0].end - gg[0].start) >= 0.005:
            peng.append(gg)
    #除去note中start和end一样的元素
    for hi, hello in enumerate(peng):
        hello.append(hi)
        print(hi, hello)
    md_li = []
    #list种要添加的顺序是：note序号，前一个音素，当前音素，下一个音素，前一个pitch，当前pitch，下一个pitch，上一个duration，当前duration，下一个duration
    for hei,kaka in enumerate(peng):
        #特殊处理第一个note
        if kaka[2] == 0:
            if peng[0][0].end == peng[1][0].start:
                md_li.append([kaka[2], 'None', kaka[1], peng[1][1], 'None', kaka[0].pitch, peng[1][0].pitch, 'None', (kaka[0].end - kaka[0].start), (peng[1][0].end - peng[1][0].start), kaka[0].start, kaka[0].end])
            else:
                md_li.append([kaka[2], 'None', kaka[1], 'None', 'None', kaka[0].pitch, 'None', 'None', (kaka[0].end - kaka[0].start), 'None', kaka[0].start, kaka[0].end])
        #特殊处理最后一个note
        elif kaka[2] == (len(peng) - 1):
            if peng[len(peng) - 1][0].start == peng[len(peng) - 2][0].end:
                md_li.append([kaka[2], peng[len(peng) - 2][1], kaka[1], 'None', peng[len(peng) - 2][0].pitch, kaka[0].pitch, 'None', (peng[len(peng) - 2][0].end - peng[len(peng) - 2][0].start), (kaka[0].end - kaka[0].start), 'None', kaka[0].start, kaka[0].end])
            else:
                md_li.append([kaka[2], 'None', kaka[1], 'None', 'None', kaka[0].pitch, 'None', 'None', (kaka[0].end - kaka[0].start), 'None', kaka[0].start, kaka[0].end])
        else:
            #与上一个note和下一个note都相连，处理方案:
            if kaka[0].start == peng[hei - 1][0].end and kaka[0].end == peng[hei + 1][0].start:
                md_li.append([kaka[2], peng[hei - 1][1], kaka[1], peng[hei + 1][1], peng[hei - 1][0].pitch, kaka[0].pitch, peng[hei + 1][0].pitch, (peng[hei - 1][0].end - peng[hei - 1][0].start), (kaka[0].end - kaka[0].start), (peng[hei + 1][0].end - peng[hei + 1][0].start), kaka[0].start, kaka[0].end])
            #与上一个note相连，但下一个note处为'None'，处理方案：
            elif kaka[0].start == peng[hei - 1][0].end and kaka[0].end != peng[hei + 1][0].start:
                md_li.append([kaka[2], peng[hei - 1][1], kaka[1], 'None', peng[hei - 1][0].pitch, kaka[0].pitch, 'None', (peng[hei - 1][0].end - peng[hei - 1][0].start), (kaka[0].end - kaka[0].start), 'None', kaka[0].start, kaka[0].end])
            #上一个note处为'None'，与下一个note相连，处理方案：
            elif kaka[0].start != peng[hei - 1][0].end and kaka[0].end == peng[hei + 1][0].start:
                md_li.append([kaka[2], 'None', kaka[1], peng[hei + 1][1], 'None', kaka[0].pitch, peng[hei + 1][0].pitch, 'None', (kaka[0].end - kaka[0].start), (peng[hei + 1][0].end - peng[hei + 1][0].start), kaka[0].start, kaka[0].end])
            elif kaka[0].start != peng[hei - 1][0].end and kaka[0].end != peng[hei + 1][0].start:
                md_li.append([kaka[2], 'None', kaka[1], 'None', 'None', kaka[0].pitch, 'None', 'None', (kaka[0].end - kaka[0].start), 'None', kaka[0].start, kaka[0].end])
    for ik, ka in enumerate(md_li):
        pp_q = ka[7]
        pp_d = ka[8]
        pp_x = ka[9]
        if pp_q != 'None':
            md_li[ik][7] = round(pp_q, 3)
        if pp_d != 'None':
            md_li[ik][8] = round(pp_d, 3)
        if pp_x != 'None':
            md_li[ik][9] = round(pp_x, 3)
    md_npy = np.array(md_li)
    frames_num = int((md_li[-1][-1])/0.005)
    print(frames_num)

    fr_li = []
    for fr in range(frames_num):
        fr_s = fr*0.005
        fr_e = (fr+1)*0.005
        for tt in md_npy:
            if float(tt[-2]) <= fr_s and float(tt[-1]) >= fr_e:
                fr_li.append(tt[0:-2])
                break
        if len(fr_li) == fr :
            fr_li.append(['None','None','None','None','None','None','None','None','None','None'])
    fr_npy = np.array(fr_li)
    print(len(fr_npy))
    return fr_npy

In [23]:
midi_folder = 'C:/Users/58450/My Code/data_process/data/midi'

npy_folder = 'C:/Users/58450/My Code/data_process/data/npy'

output_folder = 'C:/Users/58450/My Code/data_process/data/process_data'

for file in os.listdir(midi_folder):
    midi_file = os.path.join(midi_folder, file)
    print(file[:-4])
    file_1 = file[:-3] + 'npy'
    npy_file = os.path.join(npy_folder, file_1)
    print(file_1)
    np.save(output_folder + '/{}_p.npy'.format(file[:-4]), data_process(midi_file, npy_file))

1
1.npy
0 [Note(start=17.805000, end=17.945000, pitch=50, velocity=127), 'uei', 0]
1 [Note(start=17.945000, end=18.135000, pitch=52, velocity=127), 'uei', 1]
2 [Note(start=18.135000, end=18.150000, pitch=52, velocity=127), 'an', 2]
3 [Note(start=18.150000, end=18.320000, pitch=54, velocity=127), 'an', 3]
4 [Note(start=18.320000, end=18.415000, pitch=54, velocity=127), 'sh', 4]
5 [Note(start=18.415000, end=18.430000, pitch=56, velocity=127), 'sh', 5]
6 [Note(start=18.430000, end=18.560000, pitch=56, velocity=127), 'iii', 6]
7 [Note(start=18.560000, end=18.680000, pitch=56, velocity=127), 'sh', 7]
8 [Note(start=18.680000, end=18.810000, pitch=56, velocity=127), 'iii', 8]
9 [Note(start=18.810000, end=18.845000, pitch=55, velocity=127), 'iii', 9]
10 [Note(start=18.845000, end=18.910000, pitch=55, velocity=127), 'j', 10]
11 [Note(start=18.910000, end=18.925000, pitch=56, velocity=127), 'j', 11]
12 [Note(start=18.925000, end=19.155000, pitch=56, velocity=127), 'ian', 12]
13 [Note(start=19.15

17301
10
10.npy
0 [Note(start=16.620000, end=16.790000, pitch=52, velocity=127), 'uei', 0]
1 [Note(start=16.790000, end=16.855000, pitch=53, velocity=127), 'j', 1]
2 [Note(start=16.855000, end=17.035000, pitch=56, velocity=127), 'in', 2]
3 [Note(start=17.035000, end=17.080000, pitch=56, velocity=127), 'n', 3]
4 [Note(start=17.080000, end=17.085000, pitch=59, velocity=127), 'n', 4]
5 [Note(start=17.085000, end=17.295000, pitch=59, velocity=127), 'i', 5]
6 [Note(start=17.295000, end=17.305000, pitch=59, velocity=127), 'b', 6]
7 [Note(start=17.305000, end=17.330000, pitch=56, velocity=127), 'b', 7]
8 [Note(start=17.330000, end=17.415000, pitch=56, velocity=127), 'ian', 8]
9 [Note(start=17.415000, end=17.805000, pitch=56, velocity=127), 'ian', 9]
10 [Note(start=17.805000, end=17.875000, pitch=56, velocity=127), 'd', 10]
11 [Note(start=17.875000, end=18.065000, pitch=56, velocity=127), 'e', 11]
12 [Note(start=18.065000, end=18.480000, pitch=53, velocity=127), 'e', 12]
13 [Note(start=18.7350

13972
11
11.npy
0 [Note(start=2.270000, end=2.335000, pitch=59, velocity=127), 'n', 0]
1 [Note(start=2.335000, end=2.360000, pitch=60, velocity=127), 'n', 1]
2 [Note(start=2.360000, end=2.495000, pitch=60, velocity=127), 'v', 2]
3 [Note(start=2.495000, end=2.600000, pitch=60, velocity=127), 'sh', 3]
4 [Note(start=2.600000, end=2.625000, pitch=61, velocity=127), 'sh', 4]
5 [Note(start=2.625000, end=2.810000, pitch=61, velocity=127), 'eng', 5]
6 [Note(start=2.810000, end=2.860000, pitch=61, velocity=127), 'ai', 6]
7 [Note(start=2.860000, end=3.075000, pitch=57, velocity=127), 'ai', 7]
8 [Note(start=3.075000, end=3.135000, pitch=57, velocity=127), 'm', 8]
9 [Note(start=3.135000, end=3.275000, pitch=57, velocity=127), 'ei', 9]
10 [Note(start=3.275000, end=3.305000, pitch=54, velocity=127), 'ei', 10]
11 [Note(start=3.305000, end=3.345000, pitch=54, velocity=127), 'd', 11]
12 [Note(start=3.345000, end=3.440000, pitch=54, velocity=127), 'e', 12]
13 [Note(start=3.440000, end=3.485000, pitch=54

10751
12
12.npy
0 [Note(start=13.015000, end=13.380000, pitch=51, velocity=127), 'ia', 0]
1 [Note(start=13.380000, end=13.495000, pitch=51, velocity=127), 'd', 1]
2 [Note(start=13.495000, end=13.505000, pitch=56, velocity=127), 'd', 2]
3 [Note(start=13.505000, end=13.690000, pitch=56, velocity=127), 'e', 3]
4 [Note(start=13.690000, end=14.460000, pitch=58, velocity=127), 'e', 4]
5 [Note(start=14.460000, end=14.640000, pitch=58, velocity=127), 'j', 5]
6 [Note(start=14.640000, end=14.655000, pitch=58, velocity=127), 'in', 6]
7 [Note(start=14.655000, end=14.795000, pitch=58, velocity=127), 'in', 7]
8 [Note(start=14.795000, end=15.030000, pitch=60, velocity=127), 'in', 8]
9 [Note(start=15.030000, end=15.160000, pitch=60, velocity=127), 't', 9]
10 [Note(start=15.165000, end=16.675000, pitch=60, velocity=127), 'ou', 10]
11 [Note(start=17.460000, end=17.680000, pitch=60, velocity=127), 'ch', 11]
12 [Note(start=17.680000, end=17.945000, pitch=60, velocity=127), 'uei', 12]
13 [Note(start=17.945

18153
13
13.npy
0 [Note(start=11.985000, end=12.150000, pitch=59, velocity=127), 'z', 0]
1 [Note(start=12.150000, end=12.175000, pitch=59, velocity=127), 'ai', 1]
2 [Note(start=12.175000, end=12.315000, pitch=61, velocity=127), 'ai', 2]
3 [Note(start=12.315000, end=13.410000, pitch=63, velocity=127), 'ai', 3]
4 [Note(start=13.600000, end=13.630000, pitch=63, velocity=127), 'l', 4]
5 [Note(start=13.630000, end=13.690000, pitch=65, velocity=127), 'l', 5]
6 [Note(start=13.690000, end=13.745000, pitch=65, velocity=127), 'ing', 6]
7 [Note(start=13.745000, end=14.005000, pitch=66, velocity=127), 'ing', 7]
8 [Note(start=14.005000, end=14.040000, pitch=66, velocity=127), 'i', 8]
9 [Note(start=14.040000, end=14.310000, pitch=63, velocity=127), 'i', 9]
10 [Note(start=14.310000, end=14.380000, pitch=63, velocity=127), 'g', 10]
11 [Note(start=14.380000, end=14.425000, pitch=59, velocity=127), 'g', 11]
12 [Note(start=14.425000, end=14.650000, pitch=59, velocity=127), 'e', 12]
13 [Note(start=14.6500

...

49551
6
6.npy
0 [Note(start=11.380000, end=11.610000, pitch=55, velocity=127), 'ang', 0]
1 [Note(start=11.610000, end=12.390000, pitch=57, velocity=127), 'ang', 1]
2 [Note(start=12.390000, end=12.495000, pitch=57, velocity=127), 'j', 2]
3 [Note(start=12.495000, end=12.515000, pitch=57, velocity=127), 'ian', 3]
4 [Note(start=12.515000, end=12.755000, pitch=52, velocity=127), 'ian', 4]
5 [Note(start=12.755000, end=12.805000, pitch=52, velocity=127), 'd', 5]
6 [Note(start=12.805000, end=12.810000, pitch=52, velocity=127), 'd', 6]
7 [Note(start=12.810000, end=13.350000, pitch=52, velocity=127), 'e', 7]
8 [Note(start=13.350000, end=13.540000, pitch=52, velocity=127), 'j', 8]
9 [Note(start=13.540000, end=13.550000, pitch=52, velocity=127), 'in', 9]
10 [Note(start=13.550000, end=13.720000, pitch=59, velocity=127), 'in', 10]
11 [Note(start=13.720000, end=13.955000, pitch=61, velocity=127), 'in', 11]
12 [Note(start=13.955000, end=14.180000, pitch=59, velocity=127), 'in', 12]
13 [Note(start=14.1

15338
7
7.npy
0 [Note(start=18.285000, end=18.575000, pitch=55, velocity=127), 'f', 0]
1 [Note(start=18.575000, end=18.735000, pitch=55, velocity=127), 'ei', 1]
2 [Note(start=18.735000, end=18.765000, pitch=55, velocity=127), 'd', 2]
3 [Note(start=18.765000, end=18.780000, pitch=53, velocity=127), 'd', 3]
4 [Note(start=18.780000, end=18.895000, pitch=53, velocity=127), 'ao', 4]
5 [Note(start=18.895000, end=19.030000, pitch=53, velocity=127), 'ch', 5]
6 [Note(start=19.030000, end=19.040000, pitch=55, velocity=127), 'ch', 6]
7 [Note(start=19.040000, end=19.145000, pitch=55, velocity=127), 'eng', 7]
8 [Note(start=19.145000, end=19.285000, pitch=55, velocity=127), 'sh', 8]
9 [Note(start=19.285000, end=19.295000, pitch=53, velocity=127), 'sh', 9]
10 [Note(start=19.295000, end=19.450000, pitch=53, velocity=127), 'iii', 10]
11 [Note(start=19.450000, end=19.470000, pitch=53, velocity=127), 'l', 11]
12 [Note(start=19.470000, end=19.490000, pitch=55, velocity=127), 'l', 12]
13 [Note(start=19.490

21869
8
8.npy
0 [Note(start=3.375000, end=3.680000, pitch=53, velocity=127), 'iao', 0]
1 [Note(start=3.680000, end=3.815000, pitch=53, velocity=127), 'sh', 1]
2 [Note(start=3.815000, end=3.835000, pitch=51, velocity=127), 'sh', 2]
3 [Note(start=3.835000, end=4.090000, pitch=51, velocity=127), 'iii', 3]
4 [Note(start=4.090000, end=4.220000, pitch=51, velocity=127), 'h', 4]
5 [Note(start=4.220000, end=4.240000, pitch=54, velocity=127), 'h', 5]
6 [Note(start=4.240000, end=4.365000, pitch=54, velocity=127), 'ou', 6]
7 [Note(start=4.365000, end=4.715000, pitch=56, velocity=127), 'ou', 7]
8 [Note(start=4.715000, end=4.825000, pitch=54, velocity=127), 'ou', 8]
9 [Note(start=4.830000, end=4.890000, pitch=53, velocity=127), 'n', 9]
10 [Note(start=4.890000, end=5.375000, pitch=53, velocity=127), 'a', 10]
11 [Note(start=5.375000, end=5.490000, pitch=53, velocity=127), 'k', 11]
12 [Note(start=5.490000, end=5.670000, pitch=51, velocity=127), 'e', 12]
13 [Note(start=5.670000, end=5.860000, pitch=51,

15716
9
9.npy
0 [Note(start=7.900000, end=8.230000, pitch=72, velocity=127), 'e', 0]
1 [Note(start=8.230000, end=9.515000, pitch=73, velocity=127), 'e', 1]
2 [Note(start=9.515000, end=9.665000, pitch=71, velocity=127), 'e', 2]
3 [Note(start=9.665000, end=9.915000, pitch=69, velocity=127), 'e', 3]
4 [Note(start=9.915000, end=10.105000, pitch=71, velocity=127), 'e', 4]
5 [Note(start=10.105000, end=10.360000, pitch=73, velocity=127), 'e', 5]
6 [Note(start=10.360000, end=10.565000, pitch=76, velocity=127), 'e', 6]
7 [Note(start=10.565000, end=10.780000, pitch=73, velocity=127), 'e', 7]
8 [Note(start=10.780000, end=12.275000, pitch=71, velocity=127), 'e', 8]
9 [Note(start=12.275000, end=12.795000, pitch=69, velocity=127), 'e', 9]
10 [Note(start=15.160000, end=15.200000, pitch=69, velocity=127), 'n', 10]
11 [Note(start=15.200000, end=15.255000, pitch=59, velocity=127), 'n', 11]
12 [Note(start=15.255000, end=15.450000, pitch=59, velocity=127), 'i', 12]
13 [Note(start=15.450000, end=15.480000,

271 [Note(start=57.860000, end=57.920000, pitch=61, velocity=127), 'b', 271]
272 [Note(start=57.920000, end=58.260000, pitch=64, velocity=127), 'u', 272]
273 [Note(start=58.260000, end=58.315000, pitch=64, velocity=127), 'h', 273]
274 [Note(start=58.315000, end=58.350000, pitch=69, velocity=127), 'h', 274]
275 [Note(start=58.350000, end=58.525000, pitch=69, velocity=127), 'uei', 275]
276 [Note(start=58.525000, end=59.030000, pitch=71, velocity=127), 'uei', 276]
277 [Note(start=59.030000, end=59.705000, pitch=71, velocity=127), 'uei', 277]
278 [Note(start=59.880000, end=59.945000, pitch=71, velocity=127), 'z', 278]
279 [Note(start=59.945000, end=60.015000, pitch=67, velocity=127), 'z', 279]
280 [Note(start=60.015000, end=60.160000, pitch=67, velocity=127), 'ai', 280]
281 [Note(start=60.160000, end=60.360000, pitch=68, velocity=127), 'ai', 281]
282 [Note(start=60.360000, end=60.410000, pitch=68, velocity=127), 'c', 282]
283 [Note(start=60.410000, end=60.450000, pitch=69, velocity=127), '